In [13]:
# Import denpendencies
from pathlib import Path
import pandas as pd
import os

In [15]:
# reading files in dataframe
# create empty dictionary to hold data
store_data = {}
# define file paths
# define file path
file_paths = {"train":"../Resources/test.csv",
            "holidays_events":"../Resources/holidays_events.csv",
            "oil":"../Resources/oil.csv",
            "stores":"../Resources/stores.csv",
            "transactions":"../Resources/transactions.csv"}
# read and store each file data in the store_data list.
for data_file,relative_path in file_paths.items():
    # create absolute path for each file
    file_path = os.path.abspath(relative_path)
    # read and store file in dictionary
    try:
        store_data[data_file] = pd.read_csv(file_path, low_memory=False)
        print(f'{data_file} loaded successfully.')
    except FileNotFoundError:
        print(f"Error: {data_file} not found.")
    except Exception as e:
        print(f"An error occured while loading data{e}")
    

train loaded successfully.
holidays_events loaded successfully.
oil loaded successfully.
stores loaded successfully.
transactions loaded successfully.
